## Import der benötigten Bibliotheken

In [1]:
import pprint
import nltk
import csv
import pandas    as pd
from pymongo     import MongoClient
from nltk        import word_tokenize
from nltk.corpus import stopwords

## Laden der Stopwords
Die 1. Zelle muss nur beim ersten Mal ausgeführt werden

In [ ]:
nltk.download('stopwords')

In [2]:
stop = set(stopwords.words('german'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

## Konvertierung der .txt-Datenquellen in .csv-Dateien
Muss nur beim ersten Mal ausgeführt werden

In [3]:
sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', header = None, delimiter='\t')
sentences_csv.to_csv ('sentences_de.csv', index=None)

## Entfernen der Stopwords

In [4]:
def remove_stopwords(sentences):
    queue = []
    for sentence in sentences:
        result = ' '.join([i for i in sentence.lower().split() if i not in stop]) #Habe hier eine Liste statt einer Menge genommen, da sonst die Reihenfolge geändert wird
        queue.append(result)
    return queue

In [5]:
nostop = remove_stopwords(sentences_csv[1]) #(sentences_csv[1][100:200]) Für Test nur 100 Datensätze verwenden
df = pd.DataFrame(nostop)
df.to_csv('sentences_preprocessed.csv', index=False)

## Verbindung mit mongoDB

In [6]:
connection_string      = "mongodb://localhost:27017"
db                     = MongoClient(connection_string)
sentences              = db.dw.sentences
sentences_preprocessed = db.dw.sentences_preprocessed

## Sentences und Stopwords der Database in mongoDB hinzufügen

`read_csv()` liest eine .csv-Datei ein und gibt ein Dictionary zurück, dass dann mit `save_to_mongo()` in einer Collection gespeichert wird. Dafür kann zunächst die Collection gelöscht werden (`drop_collection = True`). Sonst werden die Daten zusätzlich in die Collection eingefügt.

In [7]:
def read_csv(path, columns):
    reader = open(path, 'r', encoding='utf-8')
    return csv.DictReader(reader, columns)   

In [8]:
def save_to_mongo(collection, path, columns, drop_collection):
    if drop_collection:
        collection.drop()
    data = read_csv(path, columns)
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [9]:
save_to_mongo(sentences, 'sentences_de.csv', ('id','sentence'), True)

2646184 rows are saved to "sentences" collection successfully!


In [11]:
save_to_mongo(sentences_preprocessed, 'sentences_preprocessed.csv', ('s'), True)

101 rows are saved to "sentences_preprocessed" collection successfully!


Wenn ich die als Column-Name gleich `sentence` genommen habe, dann hat er daraus eine Spalte `s`, eine `e`, eine `n` und so weiter gemacht...Deswegen hier mit der späteren Umbenennung.

In [12]:
sentences_preprocessed.update_many( {}, { '$rename': { "s": "sentence" } } )